In [51]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

yolosac = pd.read_csv('yolosac3.csv')
# yolosac3 has multiple listings of the same business
yolosac.drop_duplicates(subset=['gmap_id'], inplace=True)
#yolosac.set_index('gmap_id', inplace=True)
yolosac.drop(yolosac.columns[0], axis=1, inplace=True)


In [5]:
yolosac['category'] = yolosac['category'].astype(str)

In [2]:
categories = print(yolosac['category'].explode().value_counts())

category
restaurant    3559
auto          2070
retail        1762
beauty        1281
recreation    1210
health         745
grocery        450
education      353
Name: count, dtype: int64


In [4]:
with open('yolosac-reviews.json', 'r') as infile, open('data/ys-reviews-with-categories.json', 'w') as outfile:
  lines = infile.readlines()
  lines[0]
  '''
  for line in lines:
    data = json.load(line)
    print(data['gmap_id'])
    data['name'] = yolosac.loc[yolosac['gmap_id'] == data['gmap_id']]['category']

    outfile.write(json.dumps(data))
    '''

### THIS CODE BLOCK WORKS (I think)

Takes 230 minutes to run.

In [78]:
# Open the file and load the JSON array
with open('data/yolosac-reviews.json', 'r') as file:
    data = json.load(file)

remove = False
prev_id = ''
category = ''
# Reviews in file are ordered by business already. This should speed up category assignment.
for review in reversed(data):
    # if review has different id than previous
    if review['gmap_id'] != prev_id:
        prev_id = review['gmap_id']   
        try:
            # try to lookup category
            category = yolosac[yolosac['gmap_id'] == review['gmap_id']]['category'].iloc[0]
            review['category'] = category
            remove = False
        except IndexError:
            data.remove(review)
            remove = True
    else:
        if remove:
            data.remove(review)
        else:
            review['category'] = category

print(len(data))
with open('data/ys-reviews-with-categories.json', 'w') as file:
    json.dump(data, file)
    file.close()

1275985


In [66]:
yolosac[yolosac['gmap_id'] == '0x809ad73dbbfe5ef1:0x45f57e4e50d4c232']

,name,gmap_id,category,avg_rating,num_of_reviews
4,Caballo Bronco Barber Styling,0x809ad73dbbfe5ef1:0x45f57e4e50d4c232,beauty,4.0,28


In [74]:
category = yolosac[yolosac['gmap_id'] == '0x809ad73dbbfe5ef1:0x45f57e4e50d4c232']['category'].iloc[0]


In [59]:
# Open the file and load the JSON array
with open('data/yolosac-reviews.json', 'r') as file:
    data = json.load(file)

#data = data[:10]

# Iterate through the list to access individual objects
for review in reversed(data):
    category = yolosac['category'].loc[yolosac['gmap_id'] == review['gmap_id']]
    if category.empty:
        data.remove(review)
    else:
        review['category'] = category[0]

with open('data/ys-reviews-with-categories.json', 'w') as file:
    json.dump(data, file)
    file.close()

KeyError: 0